In [ ]:
import siibra
from nilearn import plotting

In [ ]:
difumo64 = siibra.parcellations["difumo 64"]
funcmap = difumo64.get_map("mni 152",maptype=siibra.MapType.CONTINUOUS)

In [ ]:
jubrain = siibra.parcellations["julich-brain 2.5"]
cytomap = jubrain.get_map("mni 152",maptype=siibra.MapType.CONTINUOUS)

In [ ]:
funcindex = 1
funcregionmap = funcmap.fetch(mapindex=funcindex)
funcregion = funcmap.decode_label(funcindex)
plotting.plot_stat_map(funcregionmap,title=f"Source: {funcregion.name}")

assignments = cytomap.overlap(funcregionmap)
for index,region,value in assignments:
    if value>0.01:
        plotting.plot_stat_map(
            cytomap.fetch(mapindex=index),
            title=f"Target: {region.name} ({value*100:.1f}%)")